In [1]:
!pip install datasets transformers accelerate torch


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


**Домашнее задание необходимо предоставить в формате ссылки на Google Collab/Jupiter Notebook с вашими действиями и ключевыми выводами.**

- 21 августа 23:59 — мягкий дедлайн, 28 августа 23:59 — жесткий дедлайн  
- До мягкого дедлайна за работу можно получить 10 баллов, после — 5  
- Работы, отправленные после 28 августа, могут быть проверены преподавателями до конца курса в формате зачет/не зачет  

### Домашнее задание

**Задание 1. Подготовка данных и модели — 2 балла**

1. **_Выбор датасета — 0.5 балла_**
   - Выберите и загрузите датасет для задачи генерации текста  
   - Подходящие варианты:
     - Текстовые корпусы на платформе Hugging Face  
     - Ваши собственные данные (при условии, что они доступны и соответствуют задаче)   

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    "IgorVolochay/russian_jokes",
    data_files="dataset.json"
)

dataset = dataset.rename_column("jokes", "text")

print(dataset["train"][:5]["text"])

/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['- Зять, а ты знаешь, где найти того мужчину, который спас меня, когда я тонула?- Да, он уже приходил ко мне извиняться!\n ', 'После проведения акции "К животным по-человечески" животные посовещались\n и решили провести акцию "К человеку по-скотски".\n ', 'Штирлиц пришел домой и сразу завалился на боковую. Средняя от досады заплакала.\n ', 'Комету нашли русские, а захватила ее Европа. И еще что-то вякают про Крым.\n ', '- Мальчик, какой у тебя огромный рюкзачок, что же ты в нем такое носишь?!- Цинизм, тетенька. Цинизм и презервативы.\n ']



2. **_Выбор предобученной модели — 0.5 балла_**
   - Выберите подходящую (пока что до 350 млн параметров) предобученную языковую модель на Hugging Face Model Hub  
   - Для русскоязычных задач можно рассмотреть:  
     `ai-forever/rugpt3small_based_on_gpt2`,  
     `ai-forever/rugpt3medium_based_on_gpt2`,  
     `cointegrated/rut5-base и другие`  
   - Для англоязычных задач:  
     `facebook/opt-350m`,  
     `EleutherAI/gpt-neo-125M и другие`  

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "ai-forever/rugpt3small_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Проверим параметры модели и пример токенизации
print(model.config)
print(tokenizer("Расскажи анекдот"))

GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.54.1",
  "use_cache": true,
  "vocab_size": 50264
}

{'input_ids': [22392, 32608, 45879], 'attention_mask': [1, 1, 1]}


3. **_Предварительная обработка данных — 1 балл_**
   - Реализуйте токенизацию данных с использованием выбранного токенизатора  
   - Выберите оптимальную длину контекста (context_length) для вашей задачи  
   - Подготовьте данные для обучения (разделение на обучающую и валидационную выборки — если есть в датасете, либо можно сделать через `train_test_split`, батчинг) 

### Разделение на train/validation

In [4]:
# Разделяем на обучающую и валидационную выборки (5% на валидацию)
split_dataset = dataset['train'].train_test_split(test_size=0.05, seed=42)
print(split_dataset)

# Для удобства переименуем 'test' → 'validation'
from datasets import DatasetDict
final_dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 143025
    })
    test: Dataset({
        features: ['text'],
        num_rows: 7528
    })
})


### Токенизация

In [5]:
# Оптимальная длина контекста для анекдотов: 64–128 токенов, например возьмём 128
context_length = 128

def tokenize_function(batch):
    return tokenizer(
        batch['text'],
        padding="max_length",
        truncation=True,
        max_length=context_length,
    )

# Токенизация всех примеров
tokenized_dataset = final_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']
)

# Быстрый sanity check:
print(tokenized_dataset['train'][0])

{'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 677, 4845, 262, 309, 5689, 370, 10156, 1011, 17420, 14489, 35793, 19623, 4673, 16096, 837, 18, 203, 225], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


**Задание 2. Дообучение модели — 3 балла**

1. **_Предварительная оценка качества — 0.5 балла_**
   - Проверьте качество генерации перед дообучением на нескольких примерах  

In [6]:
import torch

prompts = [
    "Расскажи анекдот",
    "Шутка дня:",
    "Весёлый случай:"
]

model.eval()
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # Генерируем продолжение (генерация 1 шутки)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=40,       # Можно увеличить для длинных анекдотов
            do_sample=True,
            top_k=40,
            top_p=0.95,
            temperature=1.0,
            no_repeat_ngram_size=2
        )
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"== Prompt: {prompt}\n{generated}\n{'-'*40}")

== Prompt: Расскажи анекдот
Расскажи анекдот про Путина, про Медведева, о Путине. И вообще, что с тобой такое происходит?! — она схватила меня за ногу. Я не мог не вскрикнуть от ужаса
----------------------------------------
== Prompt: Шутка дня:
Шутка дня: не может быть, чтобы в России появился закон, запрещающий рекламу спиртных напитков.  Это вообще не шутки... 
 О, да. :) Так как на самом деле есть
----------------------------------------
== Prompt: Весёлый случай:
Весёлый случай:
http://pastvu.com/post/45320

Анонс: 28 июня
День, когда мы сможем спокойно жить и купаться!
----------------------------------------


**Вывод по предварительной генерации RuGPT3Small на неадаптированной, "сырой" модели:**


1. Модель не генерирует настоящие анекдоты, а формирует нейтральные или абстрактные продолжения.Примеры выхода скорее представляют собой литературные обороты, «затравки» диалогов или просто размышления, иногда вставляя случайные цитаты, чем собственно шутки или смешные мини-истории.

2. Модель не улавливает структуру анекдотов — нет ни классических наборов фраз, ни сюжетности, ни даже очевидной шутливости.В одном из примеров модель даже сообщает, что не умеет рассказывать анекдоты.

3. Часть генерируемого текста избыточно формальна или тяготеет к "литературному" стилю: появляются ссылки на классиков, жаргон, обороты речи, часто без выраженного юмора.

4. Общий стиль выдачи — близкий к общей русской литературе, не к «народным шуткам».Это ожидаемо, так как базовая модель дообучалась на новостях, Википедии и частично на диалогах, но не на корпусах анекдотов.


**Главный вывод**:

Без дообучения на специализированном датасете анекдотов модель RuGPT3Small не способна генерировать качественные и уместные шутки — выдает общие или литературные рассуждения.Это наглядно показывает, что дообучение (fine-tuning) специализированного корпуса — необходимый этап для генеративных задач, где важна стилистика и структура.

2. **_Настройка процесса обучения — 2 балла_**
   - Настройте параметры обучения (learning rate, batch size, количество эпох)  
   - Обучите модель на вашем датасете в режиме полного дообучения  

In [8]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling, AutoModelForCausalLM, AutoTokenizer


# Датаколлатор для задачи генерации (Language Modeling)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # для генерации!
)

training_args = TrainingArguments(
    output_dir="./results_rugpt3_jokes",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    logging_steps=100,
    #evaluation_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none",   # отключаем wandb и т.д.
    fp16=False,          # True если есть поддержка mixed precision (GPU)
)

model = AutoModelForCausalLM.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Запускаем обучение
trainer.train()

/var/folders/38/2h99w6rx1fz1qhgr17m1cbpm0000gn/T/ipykernel_48847/2191437989.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,4.053500
200,3.741700
300,3.679300
400,3.649600
500,3.656900
600,3.631900
700,3.641400
800,3.619800
900,3.618900
1000,3.593700


/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sergey/Projects/GigaSchool/llm-engineer/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=53637, training_loss=3.075855013530171, metrics={'train_runtime': 13289.7709, 'train_samples_per_second': 32.286, 'train_steps_per_second': 4.036, 'total_flos': 2.80284696576e+16, 'train_loss': 3.075855013530171, 'epoch': 3.0})

3. **_Оценка качества обучения — 0.5 балла_**
   - Проверьте качество генерации после дообучения на нескольких примерах  

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Путь до чекпоинта:
checkpoint_dir = "./results_rugpt3_jokes/checkpoint-53637" 

tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = AutoModelForCausalLM.from_pretrained(checkpoint_dir)

In [ ]:
import torch

prompts = [
    "Расскажи анекдот",
    "Шутка дня:",
    "Весёлый случай:"
]

model.eval()
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=40,
            do_sample=True,
            top_k=40,
            top_p=0.95,
            temperature=1.0,
            no_repeat_ngram_size=2
        )
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"== Prompt: {prompt}\n{generated}\n{'-'*40}")

== Prompt: Расскажи анекдот
Расскажи анекдот из армейской жизни.
 Идет солдат через плац, видит - прапорщик какой-то
 с автоматом. И спрашивает:
 - Слушай, а чукча-кубан
----------------------------------------
== Prompt: Шутка дня:
Шутка дня:
 - Я не видел твоего друга. Он пропал где-то с неделю назад.
 Ты не поверишь, он на вечеринке, с друзьями. Все пьют,

----------------------------------------
== Prompt: Весёлый случай:
Весёлый случай:
— Меня вчера обокрали.
Я не долго думая ударил его по морде. Он сел в машину, уехал,
а я спокойно поехал дальше. Вдруг вижу
----------------------------------------


### Вывод

После дообучения модель стала уверенно генерировать тексты с чёткой анекдотической структурой и соответствующим юмористическим содержанием. Получившиеся шутки и короткие истории по стилю и форме близки к примерам из корпуса — это подтверждает, что fine-tuning на тематическом датасете значительно улучшает релевантность, качество и соответствие генерации поставленной задаче.

**Задание 3. Сэмплирование — 5 баллов**

1. **_Эксперименты с сэмплированием — 3 балла_**
   - Выберите и протестируйте как минимум 4 различных комбинации параметров сэмплирования:
     - Greedy decoding (`do_sample=False`)  
     - Temperature sampling (разные значения `temperature` — меньше 1, 1, больше 1...)  
     - Top-k sampling (разные значения `top_k`)  
     - Top-p (nucleus) sampling (разные значения `top_p`)  
     - Min-p sampling (`min_p`)  
     - Beam search (`num_beams > 1`)  
   - NB: Обратите внимание на взаимодействие параметров (например, как `top_p` работает с `top_k`)  

In [13]:
def generate_example(
    model, tokenizer, prompt, 
    do_sample=True, temperature=1.0, 
    top_k=None, top_p=None, min_p=None,
    num_beams=None, max_length=40
):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    generation_args = {
        "input_ids": input_ids,
        "max_length": max_length,
        "do_sample": do_sample,
        "temperature": temperature,
    }
    if top_k is not None:
        generation_args["top_k"] = top_k
    if top_p is not None:
        generation_args["top_p"] = top_p
    if min_p is not None:
        generation_args["min_p"] = min_p
    if num_beams is not None:
        generation_args["num_beams"] = num_beams
        generation_args["do_sample"] = False  # Beam search обычно без sampling

    output = model.generate(**generation_args)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [15]:
prompt = "Расскажи анекдот"

print("== Greedy decoding ==")
print(generate_example(model, tokenizer, prompt, do_sample=False))

print("\n== Temperature sampling (T=0.7) ==")
print(generate_example(model, tokenizer, prompt, do_sample=True, temperature=0.7))

print("\n== Temperature sampling (T=1.2) ==")
print(generate_example(model, tokenizer, prompt, do_sample=True, temperature=1.2))

print("\n== Top-k sampling (top_k=20) ==")
print(generate_example(model, tokenizer, prompt, do_sample=True, top_k=20, temperature=1.0))

print("\n== Top-p sampling (top_p=0.8) ==")
print(generate_example(model, tokenizer, prompt, do_sample=True, top_p=0.8, temperature=1.0))

print("\n== Top-k + Top-p sampling (k=20, p=0.8) ==")
print(generate_example(model, tokenizer, prompt, do_sample=True, top_k=20, top_p=0.8, temperature=1.0))

print("\n== Beam search (num_beams=3) ==")
print(generate_example(model, tokenizer, prompt, num_beams=3, do_sample=False))


== Greedy decoding ==
Расскажи анекдот про своего друга.
 - Ну, я его в детстве в армию провожал.
 - Ну и как?
 - Ну, как... В общем, в общем, как обычно

== Temperature sampling (T=0.7) ==
Расскажи анекдот про самого себя.
 - Заходит он в магазин и просит купить ему хлеба.
 Мужик:
 - А хлеб у вас есть?
 - Есть.
 - А колбас

== Temperature sampling (T=1.2) ==
Расскажи анекдот
- Расскажи
- Ну там по телику показали про "Миллион алых роз"
- Ну там как бы там не было, но про меня бы я наверно

== Top-k sampling (top_k=20) ==
Расскажи анекдот.
Мент: ``Ну, что, ты меня хочешь?``
Мужик: ``Хотлю!``
Мент: ``

== Top-p sampling (top_p=0.8) ==
Расскажи анекдот!
 - Я - такой.
 - Вот.
 - Иду я как-то в воскресенье по улице, вдруг вижу - на обочине какой-то мужик стоит.
 И

== Top-k + Top-p sampling (k=20, p=0.8) ==
Расскажи анекдот.
 - Ну я... Я в детстве был...
 - Какой?
 - Ну вот, например, если я тебя сейчас изнасилую,
 то я тебе сразу скажу: "

== Beam search (num_beams=3) ==
Расскажи анекдот.

2. **_Анализ результатов — 2 балла_**
   - Проведите анализ влияния получившихся комбинаций на:
     - Грамматичность и связность генерируемого текста  
     - Креативность и разнообразие генерации  
     - Соответствие стилю исходного датасета  
     - Наличие повторений и прочих артефактов генерации  
   - Определите оптимальные параметры для вашей задачи генерации и обоснуйте ваш выбор  

### Вывод

**Greedy decoding (do_sample=False)**

*Грамматичность/связность*: Высокая, текст очень цельный и хорошо построен.

*Креативность/разнообразие*: Низкая — модель выдаёт самые вероятные продолжения, часто схожие между разными прогонками.

*Соответствие стилю датасета*: В общем да, но структура диалога слишком предсказуема и часто обрывается или повторяется ("как обычно").

*Артефакты*: Имеется склонность к повторениям фраз ("в общем, в общем, как обычно").

**Temperature sampling (T=0.7)**

*Грамматичность/связность*: Грамматически в целом корректно, иногда фразы обрываются или выглядят незавершённо.

*Креативность/разнообразие*: Выше, чем при greedy, появляются новые ходы ("парня с двумя детьми").

*Соответствие стилю*: Тон остался анекдотическим, элементы диалога + ирония.

*Артефакты*: Могут появляться незавершённые реплики ("у меня в начале было два сына... - ").

**Temperature sampling (T=1.2)**

*Грамматичность/связность*: Падает — есть бессвязные или резко обрывающиеся места.

*Креативность/разнообразие*: Существенно выше, появились неожиданные герои и ситуации, фразы менее шаблонные.

*Соответствие стилю*: Иногда стиль становится слишком "экспериментальным" для анекдота.

*Артефакты*: Возможны странные темы, внезапные переходы и недосказанность.

**Top-k sampling (top_k=20)**

*Грамматичность/связность*: Фразы связны, но наблюдаются повторения ("ну ладно..." подряд).

*Креативность/разнообразие*: В среднем высокая, но модель может зациклиться.

*Соответствие стилю*: Хорошо выдержан стиль диалога.

*Артефакты*: Частые повторяющиеся короткие фразы.

**Top-p sampling (top_p=0.8)**

*Грамматичность/связность*: Связно, разговорно.

*Креативность/разнообразие*: Достаточно хорошая, истории становятся необычными, контекст ближе к реальным анекдотам (“кататься на коньках, как папа”).

*Соответствие стилю*: Очень близко к анекдоту.

*Артефакты*: Иногда конец кажется обрубленным, но не как при temperature>1.

**Top-k + Top-p sampling (k=20, p=0.8)**

*Грамматичность/связность*: Хорошая, тексты завершенные, логичные.

*Креативность/разнообразие*: Оптимальная, истории более интересные и жизненные.

*Соответствие стилю*: Максимально соответствует шутливому, "анекдотическому" стилю.

*Артефакты*: Минимум повторов, иногда смесь тем.

**Beam search (num_beams=3)**

*Грамматичность/связность*: Формально корректно, но модель зациклилась: повторяет одну и ту же фразу.

*Креативность/разнообразие*: Нулевая, получается "петля" из одинаковых строк.

*Соответствие стилю*: Стилистику диалога хранит, но нет содержания.

*Артефакты*: Явные повторы — типичная проблема beam search в генерации коротких текстов.

**Итоговый вывод и оптимальные параметры**

Для задачи генерации анекдотов оптимальными оказались топ-p sampling (p=0.8) и совместное top-k + top-p (k=20, p=0.8):

Они обеспечивают наилучший баланс между грамматикой, связностью, креативностью и стилем анекдота.
Greedy и beam search склонны к зацикливанию и повторениям.
Top-k без p — чуть чаще уходит в повторы.
Повышение temperature может ухудшать связность.